# Import Libraries

In [16]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from scipy.signal import find_peaks
from height_experiments.tools import extract_exercises, smooth_column
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go
from height_experiments.tools import compute_height_initial_velocity

# Connect to Firebase

In [2]:
cred = credentials.Certificate('firebase_python.json')

In [3]:
firebase_admin.initialize_app(cred)

In [4]:
db = firestore.client()

In [17]:
collection = db.collection('signals_db')

In [18]:
res = collection.get() # returns a list
res = [i.to_dict() for i in res]
document = sorted(res, key=lambda x: x["info"]["date"].timestamp(), reverse=True)[0]["info"]

In [19]:
document["date"]

DatetimeWithNanoseconds(2024, 2, 21, 11, 27, 6, 550000, tzinfo=datetime.timezone.utc)

# Adding time values

In [20]:
for index in range(len(document["timeStampSamples"])):
    document["sensorAcc"][index]["time"] = document["timeStampSamples"][index]

# Smooth signal and extract exercises

In [21]:
df = pd.DataFrame(document["sensorAcc"])
df = smooth_column(df, "valueZ")
df = df.iloc[::-1].reset_index(drop=True)

# Detecting peaks
peaks, _ = find_peaks(df['valueZ_Smooth'])
exercises = extract_exercises(df.to_dict(orient='records'), peaks.tolist(), 'valueZ_Smooth')

df.head()

,valueY,valueZ,valueX,time,valueZ_Smooth
0,0.026498,-0.023058,0.031625,2649822,-0.023058
1,0.029187,-0.024646,0.002015,2649831,-0.016443
2,0.029603,0.007084,-0.008344,2649840,-0.009859
3,0.023403,-0.003374,-0.029572,2649849,-0.004749
4,0.027790,-0.009832,-0.040120,2649858,0.000161


# Show signals

In [22]:
fig = px.line(df, y='valueZ_Smooth', title='Linear Acceleration Z over Time')
fig.show()

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['valueZ_Smooth'], mode='lines', name='Time Series'))
fig.add_trace(go.Scatter(x=df.index[peaks], y=df['valueZ_Smooth'][peaks], mode='markers', marker=dict(size=10, color='red'), name='Peaks'))
fig.update_layout(title='Time Series Data with Peaks', xaxis_title='Time', yaxis_title='Value')
fig.show()

# Show last repetitions

In [24]:
for num, exercise in enumerate(exercises):
    df_ex = pd.DataFrame(exercise)
    jump_height = compute_height_initial_velocity(df_ex, "valueZ_Smooth")
    fig = px.line(df_ex, y='valueZ_Smooth', title=f'Jump #{num}: {jump_height}m')
    fig.show()

# Comparing App Results

In [25]:
jumps_flutter = document["jumpHeights"]
for num, exercise in enumerate(exercises):
    df_ex = pd.DataFrame(exercise)
    jump_height = compute_height_initial_velocity(df_ex, "valueZ_Smooth")
    jump_app = jumps_flutter[num]
    print(f"Jump #{num}: Python: {jump_height}m ; Flutter: {jump_app}")

Jump #0: Python: 0.5538879190032601m ; Flutter: 0.25771752375897194
Jump #1: Python: 0.5495666128667352m ; Flutter: 0.24771963871972386
